In [145]:
import os
os.environ['USE_PYGEOS'] = '0'

import numpy as np
import pandas as pd 
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt

from temporal_curve import temporal_curve_24hr
from temporal_curve import future_ddfs

In [146]:
ARI = 100
ens_ptl  = 5  # 50% ensemble percentile

In [147]:
# Get regional polygons
gdf_nz_bf = gpd.read_file(r"E:\Tuflow\Cal_rainfall_timeseries\data\NZ_ClimateZone\NZClimate.shp")
gdf_nz_bf


,Shape_Leng,Shape_Area,CZ,geometry
0,19.853261,6.903691,ESI,"MULTIPOLYGON (((172.69896 -43.63224, 172.68986..."
1,45.386298,8.176955,WSI,"MULTIPOLYGON (((167.39786 -47.25784, 167.39576..."
2,21.293166,2.204193,NSI,"MULTIPOLYGON (((173.17846 -41.29894, 173.17636..."
3,10.907627,3.041842,WNI,"MULTIPOLYGON (((174.78036 -41.09934, 174.77226..."
4,18.040459,3.195582,ENI,"MULTIPOLYGON (((177.86326 -39.30804, 177.86066..."
5,43.263371,5.819416,NNI,"MULTIPOLYGON (((176.97606 -37.85714, 176.96996..."


In [148]:
# Read historical regional DDF 
his_ddfs_file = r'E:\Tuflow\Cal_rainfall_timeseries\output\byNZClimate\baseline\baseline_regional_ddfs_ARI' + str(ARI) + '_byNZClimate.csv'
his_ddfs = pd.read_csv(his_ddfs_file, sep=',', index_col=0)
his_ddfs



,ESI,WSI,NSI,WNI,ENI,NNI
mins,,,,,,
10.0,18.600170,23.044006,19.928719,26.005274,22.204578,27.516214
20.0,25.661398,32.633369,29.505239,34.689045,31.456469,39.616444
30.0,31.379242,40.435131,37.358040,41.151539,38.799446,48.772942
60.0,44.910141,59.073151,56.153175,55.203884,55.794502,68.841179
120.0,64.866028,86.992645,84.106689,74.014153,80.162056,95.627319
360.0,115.183922,159.303528,154.393326,116.685364,139.519806,154.704102
720.0,161.658951,228.187942,218.123306,153.629242,193.169357,203.300415
1440.0,219.648117,316.859802,295.287354,199.278488,260.115723,259.768097
2880.0,285.414612,421.628174,378.700500,253.400986,338.100616,321.411346


In [149]:
# data_file = r"E:\Tuflow\Cal_rainfall_timeseries\data\New Zealand_Extreme_Precipitation_baseline_NIWA.nc"
# da_ddfs_100a = xr.open_dataset(data_file)['Extreme_Precipitation'].sel(
#     ARI=ARI).drop(["ARI"]).rename({"lat": "latitude", "lon":"longitude"})
# da_ddfs_100a.rio.write_crs("epsg:4326", inplace=True)
# dataset = xr.open_dataset(data_file)
# variable_names = list(dataset.variables)
# variable_names
# for var_name in ['ARI', 'Hrs']:
#     var_values = dataset[var_name].values
#     print(f"Variable: {var_name}")
#     print(f"Values: {var_values}")
#     print()

In [150]:
# read AR6 pattern
# ens_ptl  = 50  # 50% ensemble percentile
gev_file = "data/New Zealand_Extreme_Precipitation_Pattern_AR6.nc"

In [151]:
dataset = xr.open_dataset(gev_file)
variable_names = list(dataset.variables)
variable_names

['ari', 'hrs', 'pth', 'CZ', 'pattern']

In [152]:
for var_name in variable_names[:-1]:
    var_values = dataset[var_name].values
    print(f"Variable: {var_name}")
    print(f"Values: {var_values}")
    print()

Variable: ari
Values: [  2   3   5  10  15  25  50 100 150 200 300 500]

Variable: hrs
Values: [  3   6  12  24  48  72  96 120 144 240]

Variable: pth
Values: [ 1.   5.  15.9 25.  50.  75.  84.1 95.  99. ]

Variable: CZ
Values: ['ESI' 'WSI' 'NSI' 'WNI' 'ENI' 'NNI']



In [153]:
# # Read CZ='NNI' region
# da_cf = xr.open_dataset(gev_file).pattern.sel(ari=ARI, 
#                                               pth=ens_ptl, 
#                                               method='nearest'
#                                              ).drop(['ari', 'pth']).sel(CZ="NNI").drop(['CZ'])
# # to pandas.DataFrame
# df_cf = da_cf.to_dataframe().rename({"pattern":"cf"}, axis=1)
# df_cf.index = np.round(df_cf.index*60, 4) # transform duration units to minutes
# df_cf.index.name = "mins"


# Read all CZ regions
da_cf = xr.open_dataset(gev_file).pattern.sel(ari=ARI, 
                                              pth=ens_ptl, 
                                              method='nearest'
                                             ).drop(['ari', 'pth'])  
# to pandas.DataFrame
df_cf = da_cf.to_dataframe().rename({"pattern":"cf"}, axis=1)
df_cf = df_cf.rename(index=lambda x: x * 60, level='hrs')
df_cf.index.names = ['CZ', 'min']


df_cf

cf
CZ  min             
ESI 180    -5.921977
    360    -6.537296
    720    -7.152234
    1440   -7.766791
    2880   -8.380968
    4320   -8.740063
    5760   -8.994765
    7200   -9.192283
    8640   -9.353637
    14400  -9.805577
WSI 180    -6.169707
    360    -6.820445
    720    -7.470757
    1440   -8.120643
    2880   -8.770103
    4320   -9.149816
    5760   -9.419137
    7200   -9.627990
    8640   -9.798601
    14400 -10.276463
NSI 180    -6.743263
    360    -7.363932
    720    -7.984212
    1440   -8.604105
    2880   -9.223611
    4320   -9.585819
    5760   -9.842730
    7200  -10.041960
    8640  -10.204712
    14400 -10.660566
WNI 180    -6.932665
    360    -7.444285
    720    -7.955642
    1440   -8.466735
    2880   -8.977564
    4320   -9.276258
    5760   -9.488131
    7200   -9.652440
    8640   -9.786671
    14400 -10.162659
ENI 180    -7.067554
    360    -7.486074
    720    -7.904419
    1440   -8.322588
    2880   -8.740581
    4320   -8.985008
    5760   -9.158397
    7200   -9.292866
    8640   -9.402721
    14400  -9.710449
NNI 180    -6.787527
    360    -7.458686
    720    -8.129391
    1440   -8.799643
    2880   -9.469442
    4320   -9.861040
    5760  -10.138789
    7200  -10.354174
    8640  -10.530122
    14400 -11.022923

In [154]:
df_cf.loc['ESI']

,cf
min,
180,-5.921977
360,-6.537296
720,-7.152234
1440,-7.766791
2880,-8.380968
4320,-8.740063
5760,-8.994765
7200,-9.192283
8640,-9.353637


In [155]:
# Read AR6 GMT, mid
ptn_file = "data/IPCC_AR6_GMT_curves.csv"

df_dgmt = pd.read_csv(ptn_file, index_col=0)
# df_dgmt = df_dgmt.iloc[5::5, 0:len(df_dgmt.columns):3]
df_dgmt = df_dgmt.iloc[5:, 0:len(df_dgmt.columns):3]
df_dgmt.columns = [col_name.split(":")[0] for col_name in df_dgmt.columns]

df_dgmt

,SSP1-1.9,SSP1-2.6,SSP2-4.5,SSP3-7.0,SSP5-8.5
Year,,,,,
2010,0.21,0.17,0.12,0.12,0.13
2011,0.25,0.21,0.15,0.15,0.15
2012,0.28,0.24,0.18,0.17,0.18
2013,0.32,0.27,0.20,0.20,0.21
2014,0.35,0.30,0.23,0.23,0.24
...,...,...,...,...,...
2096,0.70,1.21,2.02,3.38,4.38
2097,0.70,1.21,2.02,3.43,4.45
2098,0.70,1.22,2.02,3.48,4.51


In [156]:
df_cf.loc['ESI']

,cf
min,
180,-5.921977
360,-6.537296
720,-7.152234
1440,-7.766791
2880,-8.380968
4320,-8.740063
5760,-8.994765
7200,-9.192283
8640,-9.353637


In [157]:
# df_cf_oneindex = df_cf.reset_index(level='CZ')

In [158]:
df_dgmt.columns

Index(['SSP1-1.9', 'SSP1-2.6', 'SSP2-4.5', 'SSP3-7.0', 'SSP5-8.5'], dtype='object')

In [159]:
# Iterate by SSP senarios 
fut_ddfs_full = pd.DataFrame()
fut_cdfs_full = pd.DataFrame()
fut_pdfs_full = pd.DataFrame()
    
for column_name in df_dgmt.columns:
    
    column_data = df_dgmt[column_name]
    
    for region in ['ESI', 'WSI', 'NSI', 'WNI', 'ENI', 'NNI']:
    
        fut_ddfs = pd.DataFrame()
        fut_cdfs = pd.DataFrame()
        fut_pdfs = pd.DataFrame()

        for index, row in column_data.iteritems():
            dgmt = row

            df_cf_region = df_cf.loc[region]

            his_ddfs_region = pd.DataFrame(his_ddfs[region])
            his_ddfs_region.rename(columns={region: 'ddf'}, inplace=True) 

            fut_ddfs_region = future_ddfs(dgmt, df_cf_region, his_ddfs_region)
            
            # Convert to PDF
            df_crrs_fut = temporal_curve_24hr(fut_ddfs_region)
            crrs_cdf_series_fut = pd.DataFrame(df_crrs_fut['crr'])
            crrs_pdf_series_fut = pd.DataFrame(df_crrs_fut['crr'].diff().fillna(df_crrs_fut['crr'][0]))
            
            fut_ddfs_region.rename(columns={'ddf': index}, inplace=True)
            fut_ddfs = pd.concat([fut_ddfs, fut_ddfs_region], axis=1)
            
            crrs_cdf_series_fut.rename(columns={'crr': index}, inplace=True)
            fut_cdfs = pd.concat([fut_cdfs, crrs_cdf_series_fut], axis=1)
            crrs_pdf_series_fut.rename(columns={'crr': index}, inplace=True)
            fut_pdfs = pd.concat([fut_pdfs, crrs_pdf_series_fut], axis=1)
            
            
        # ddfs
        fut_ddfs['CZ'] = region
        fut_ddfs.set_index('CZ', append=True, inplace=True)
        fut_ddfs['SSP'] = column_name
        fut_ddfs.set_index('SSP', append=True, inplace=True)

        fut_ddfs_full = fut_ddfs_full.append(fut_ddfs)
        
        
        # cdfs
        fut_cdfs['CZ'] = region
        fut_cdfs.set_index('CZ', append=True, inplace=True)
        fut_cdfs['SSP'] = column_name
        fut_cdfs.set_index('SSP', append=True, inplace=True)

        fut_cdfs_full = fut_cdfs_full.append(fut_cdfs)
        
        
        # pdfs
        fut_pdfs['CZ'] = region
        fut_pdfs.set_index('CZ', append=True, inplace=True)
        fut_pdfs['SSP'] = column_name
        fut_pdfs.set_index('SSP', append=True, inplace=True)

        fut_pdfs_full = fut_pdfs_full.append(fut_pdfs)

print('done!')

done!


In [160]:
his_ddfs_region

,ddf
mins,
10.0,27.516214
20.0,39.616444
30.0,48.772942
60.0,68.841179
120.0,95.627319
360.0,154.704102
720.0,203.300415
1440.0,259.768097
2880.0,321.411346


In [161]:
fut_ddfs_full

,,,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
mins,CZ,SSP,,,,,,,,,,,,,,,,,,,,,
10.0,ESI,SSP1-1.9,18.5,18.4,18.4,18.4,18.4,18.4,18.3,18.3,18.3,18.3,...,18.2,18.2,18.2,18.2,18.2,18.2,18.2,18.2,18.2,18.2
20.0,ESI,SSP1-1.9,25.4,25.4,25.4,25.3,25.3,25.3,25.2,25.2,25.2,25.2,...,24.9,24.9,24.9,24.9,24.9,24.9,24.9,24.9,24.9,24.9
30.0,ESI,SSP1-1.9,31.1,31.0,31.0,30.9,30.9,30.9,30.8,30.8,30.7,30.7,...,30.4,30.4,30.4,30.4,30.4,30.4,30.4,30.4,30.4,30.4
60.0,ESI,SSP1-1.9,44.4,44.4,44.3,44.2,44.1,44.1,44.0,43.9,43.9,43.8,...,43.4,43.4,43.4,43.4,43.4,43.4,43.4,43.4,43.4,43.4
120.0,ESI,SSP1-1.9,64.1,64.0,63.9,63.7,63.6,63.5,63.4,63.3,63.2,63.1,...,62.3,62.3,62.3,62.3,62.3,62.3,62.3,62.3,62.3,62.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1440.0,NNI,SSP5-8.5,256.8,256.3,255.7,255.0,254.3,253.6,252.9,252.2,251.3,250.6,...,167.0,165.4,164.0,162.6,161.0,159.6,158.0,156.7,155.1,153.7
2880.0,NNI,SSP5-8.5,317.5,316.8,315.9,315.0,314.1,313.2,312.3,311.4,310.2,309.2,...,197.8,195.7,193.9,192.1,189.9,188.1,186.0,184.1,182.0,180.2
4320.0,NNI,SSP5-8.5,353.5,352.8,351.7,350.7,349.6,348.6,347.5,346.4,345.0,344.0,...,214.7,212.3,210.1,208.0,205.5,203.4,201.0,198.8,196.4,194.2


In [162]:
fut_pdfs_full

,,,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
hours,CZ,SSP,,,,,,,,,,,,,,,,,,,,,
0.0,ESI,SSP1-1.9,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
0.1,ESI,SSP1-1.9,0.0013,0.0013,0.0013,0.0013,0.0013,0.0013,0.0013,0.0013,0.0013,0.0013,...,0.0013,0.0013,0.0013,0.0013,0.0013,0.0013,0.0013,0.0013,0.0013,0.0013
0.2,ESI,SSP1-1.9,0.0014,0.0014,0.0014,0.0014,0.0014,0.0013,0.0014,0.0014,0.0013,0.0014,...,0.0013,0.0013,0.0013,0.0013,0.0013,0.0013,0.0013,0.0013,0.0013,0.0013
0.3,ESI,SSP1-1.9,0.0014,0.0013,0.0014,0.0013,0.0013,0.0014,0.0013,0.0013,0.0014,0.0013,...,0.0014,0.0014,0.0014,0.0014,0.0014,0.0014,0.0014,0.0014,0.0014,0.0014
0.4,ESI,SSP1-1.9,0.0014,0.0014,0.0014,0.0014,0.0014,0.0014,0.0015,0.0014,0.0014,0.0014,...,0.0014,0.0014,0.0014,0.0014,0.0014,0.0014,0.0014,0.0014,0.0014,0.0014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23.6,NNI,SSP5-8.5,0.0010,0.0010,0.0011,0.0011,0.0010,0.0010,0.0010,0.0011,0.0011,0.0011,...,0.0008,0.0008,0.0007,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0007
23.7,NNI,SSP5-8.5,0.0011,0.0011,0.0010,0.0010,0.0011,0.0011,0.0011,0.0010,0.0010,0.0010,...,0.0007,0.0008,0.0008,0.0007,0.0007,0.0008,0.0007,0.0007,0.0007,0.0007
23.8,NNI,SSP5-8.5,0.0010,0.0010,0.0011,0.0011,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,...,0.0008,0.0007,0.0007,0.0008,0.0008,0.0007,0.0007,0.0007,0.0007,0.0007


In [163]:
fut_ddfs_full_file = r'E:\Tuflow\Cal_rainfall_timeseries\output\byNZClimate\future\DDFs\future_regional_ddfs_ARI' + str(ARI) + '_' + str(ens_ptl) + 'th_byNZClimate.csv'
fut_ddfs_full.to_csv(fut_ddfs_full_file, index=True, header=True, float_format='%.8f')

fut_cdfs_full_file = r'E:\Tuflow\Cal_rainfall_timeseries\output\byNZClimate\future\CDFs\future_regional_cdfs_ARI' + str(ARI) + '_' + str(ens_ptl) + 'th_byNZClimate.csv'
fut_cdfs_full.to_csv(fut_cdfs_full_file, index=True, header=True, float_format='%.8f')

fut_pdfs_full_file = r'E:\Tuflow\Cal_rainfall_timeseries\output\byNZClimate\future\PDFs\future_regional_pdfs_ARI' + str(ARI) + '_' + str(ens_ptl) + 'th_byNZClimate.csv'
fut_pdfs_full.to_csv(fut_pdfs_full_file, index=True, header=True, float_format='%.8f')


In [164]:
pdfs_df_his, pdfs_df_fut

NameError: name 'pdfs_df_his' is not defined

In [ ]:
ax = pd.concat([pdfs_df_his*his_ddfs.loc[1440].values, 
                pdfs_df_fut*fut_ddfs.loc[1440].values
                ], axis=1).plot()

In [ ]:
his_ddfs, fut_ddfs

In [ ]:
# get temporal curve from DDFs
df_crrs_his = temporal_curve_24hr(his_ddfs)
df_crrs_fut = temporal_curve_24hr(fut_ddfs)
df_crrs_his, df_crrs_fut

In [ ]:
his_ddfs

In [ ]:
his_ddfs.loc[1440]

In [ ]:
ax = pd.concat([df_crrs_his*his_ddfs.loc[1440].values, 
                df_crrs_fut*fut_ddfs.loc[1440].values
               ], axis=1).plot()
